In [1]:

# Load required libraries˜
import pandas as pd
import numpy as np

import argparse
import json
import pprint
import requests
import sys
import urllib

from __future__ import print_function

import csv
import re
import ast

In [2]:
#YELP API STARTER CODE  with edits to work with py3 and to suit my purposes

API_KEY =''

API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
SEARCH_LIMIT = 3


def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
#     url = '{0}{1}'.format(host, quote(path.encode('utf8'))) ***This seems uneccessary why? also doesnt work w py3
    url = host + path
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

#     print(u'Querying {0} ...'.format(url))
    response = requests.request('GET', url, headers=headers, params=url_params)
    return response.json()

def searchaddress(api_key, term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to t/
        he API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': 1
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def main():
    parser = argparse.ArgumentParser()

    parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM,
                        type=str, help='Search term (default: %(default)s)')
    parser.add_argument('-l', '--location', dest='location',
                        default=DEFAULT_LOCATION, type=str,
                        help='Search location (default: %(default)s)')

    input_values = parser.parse_args()

    try:
        query_api(input_values.term, input_values.location)
    except HTTPError as error:
        sys.exit(
            'Encountered HTTP error {0} on {1}:\n {2}\nAbort program.'.format(
                error.code,
                error.url,
                error.read(),
            )
        )
        
        

In [3]:
#READ RAW INSPECTION DATA 
food_inspections_orig = pd.read_csv('food_inspections.csv') 

#CLEAN INSPECTION RESULTS

    #normalize data to lower case
food_inspect_lower = food_inspections_orig.apply(lambda x: x.astype(str).str.lower()) 

    #filter to inlcude only certain business types
facility_type = 'restaurant'
food_inspect_filtered = food_inspect_lower[ food_inspect_lower['Facility Type'] == facility_type]

    #strip whitespace and reset index
food_inspect_filtered = food_inspect_filtered.apply(lambda x: x.astype(str).str.strip()) 

In [4]:
    #get only restaurants that failed inspections and reset the index
failed_food_inspect_filtered = (food_inspect_filtered[(food_inspect_filtered['Results'] == 'fail')]).reset_index(drop=True) 

    #sends failed inspection data to csv file
failed_food_inspect_filtered.to_csv("./failed_food_inspections.csv", sep =',', index=False)

In [5]:
    #create list to add yelp queries to 
bizlist =[]

In [6]:
(len(failed_food_inspect_filtered)+1)

20706

In [9]:
#0-4700 FailBizFile1
#4800-9500 FailBizFile2
#9500-14400 FailBizFile3
#14400-19200 FailBizFile4
#19200-20706 len(failed_food_inspect_filtered)+1) FailBizFile4
    
    #find and query yelp business data for restaurants that failed inspections
    #Uses restaurant name(DBA name) and address to find best match
    #queries between 2 indexes bc can only do 5000 a day
intx = 0
inty = 0
    
for i in range(intx, inty ) :
    address = failed_food_inspect_filtered['Address'][i] + ' Chicago, IL'
    name = failed_food_inspect_filtered['DBA Name'][i]
    biz = searchaddress(API_KEY, name, address)
    biz = biz.get('businesses', None)
    try:
        biz = biz[0]
        biz['FI Name'] = name
        bizlist.append(biz)
    except:
        print('Bye Felcia.') #if no result is found skips that restaurant

In [7]:
len(bizlist)

0

In [8]:
    #Send data to csv 
failedbizdf = pd.DataFrame(bizlist)
failedbizdf.to_csv("./failedBizTest.csv", sep =',', index=False)
